<a href="https://colab.research.google.com/github/thiago-leonel-lab/notebooks-google/blob/main/notebook_auditoria_52978.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho 52978 - Auditoria Assistida por IA
Este notebook realiza auditoria de fichas AVL usando IA generativa (acessada via API). Os dados de origem são da DICOI (unidade auditada), e os resultados serão salvos no repositório da AUDIT.
Organização modular para melhor manutenibiludade e facilitar a colaboração.

Nesse exemplo usamos:
Python - Google Colab;
Banco de Dados em Nuvem - Google Cloud SQL (instância Postgre);
Agente Semântico - API Gemini.

O projeto no BB deverá ser adaptado para o uso das ferramentas/tecnologias corporativas.

In [ ]:
# Bibliotecas (conferir e instalar conforme necessário)
!pip install psycopg2
!pip install pandas
!pip install requests
!pip install openai
!pip install python-dotenv

In [9]:
# Utilizar credenciais do usuário google autenticado
from google.colab import auth
auth.authenticate_user()

In [6]:
# Conectar Google Cloud
project_id = "colab-audit-agent" #@param {type:"string"}

# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"

# Configure gcloud.
!gcloud config set project {project_id}

Updated property [core/project].


In [7]:
# Garante/Confirma acesso ao papel "Cloud SQL Client" para o usuário autenticado
current_user = !gcloud auth list --filter=status:ACTIVE --format="value(account)"

!gcloud projects add-iam-policy-binding {project_id} \
  --member=user:{current_user[0]} \
  --role="roles/cloudsql.client"

Updated IAM policy for project [colab-audit-agent].
bindings:
- members:
  - user:educa.thiago.leonel@gmail.com
  role: roles/cloudsql.client
- members:
  - user:educa.thiago.leonel@gmail.com
  role: roles/cloudsql.instanceUser
- members:
  - serviceAccount:service-95842506208@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:95842506208-compute@developer.gserviceaccount.com
  - serviceAccount:95842506208@cloudservices.gserviceaccount.com
  role: roles/editor
- members:
  - serviceAccount:service-95842506208@gcp-sa-notebooks.iam.gserviceaccount.com
  role: roles/notebooks.serviceAgent
- members:
  - user:educa.thiago.leonel@gmail.com
  role: roles/owner
etag: BwY_RQ_q_oA=
version: 1


In [8]:
# Ativar a API "Cloud SQL Admin"
!gcloud services enable sqladmin.googleapis.com

###💽 **Create a Postgres Instance**
Running the below cell will verify the existence of a Cloud SQL instance or create a new one if one does not exist.

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [ ]:
#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.

# Please fill in these values.
region = "southamerica-east1" #@param {type:"string"}
instance_name = "colab-audit" #@param {type:"string"}

# Quick input validations.
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"

# check if Cloud SQL instance exists in the provided region
database_version = !gcloud sql instances describe {instance_name} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  password = input("Please provide a password to be used for 'postgres' database user: ")
  !gcloud sql instances create {instance_name} --database-version=POSTGRES_14 \
    --region={region} --cpu=1 --memory=4GB --root-password={password} \
    --database-flags=cloudsql.iam_authentication=On

In [10]:
# Criar um banco de dados na Instância PostgreSQL conectada
!gcloud sql databases create dicoi --instance=colab-audit

In [20]:
# Carregamento de variáveis de ambiente
from google.colab import userdata
import os
import psycopg2

# Conexões com Bancos de Dados
conn_dicoi = psycopg2.connect(userdata.get("BASE_DICOI"))
conn_audit = psycopg2.connect(userdata.get("BASE_AUDIT"))

# Chave da API do Agente de Auditoria
#AGENTE_API_KEY = userdata.get("OPENAI_API_KEY")

In [21]:
# Testando conexões com os Bancos de Dados
try:
    cursor_dicoi = conn_dicoi.cursor()
    cursor_dicoi.execute("SELECT 1")
    print("Conexão com BASE_DICOI bem-sucedida!")
    cursor_dicoi.close()
except Exception as e:
    print(f"Erro ao conectar com BASE_DICOI: {e}")

try:
    cursor_audit = conn_audit.cursor()
    cursor_audit.execute("SELECT 1")
    print("Conexão com BASE_AUDIT bem-sucedida!")
    cursor_audit.close()
except Exception as e:
    print(f"Erro ao conectar com SUPABASE_AUDIT_URL: {e}")

# Testando a chave da API do OpenAI
#try:
#    from openai import OpenAI
#    client = OpenAI(api_key=OPENAI_API_KEY)
#    response = client.chat.completions.create(
#        model="gpt-3.5-turbo",
#        messages=[
#            {"role": "system", "content": "You are a helpful assistant."},
#            {"role": "user", "content": "Hello, world!"},
#        ],
#        max_tokens=5
#    )
#    print("Chave OPENAI_API_KEY válida e API acessível!")
#except Exception as e:
#    print(f"Erro ao testar OPENAI_API_KEY: {e}")

Conexão com BASE_DICOI bem-sucedida!
Conexão com BASE_AUDIT bem-sucedida!


## Extração de dados da base DICOI
Seleciona os 7 campos relevantes para auditoria.

In [1]:
# install dependencies
import sys
!{sys.executable} -m pip install cloud-sql-python-connector["pg8000"] SQLAlchemy==2.0.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.43
    Uninstalling SQLAlchemy-2.0.43:
      Successfully uninstalled SQLAlchemy-2.0.43


In [2]:
# initialize parameters
INSTANCE_CONNECTION_NAME = f"{project_id}:{region}:{instance_name}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "auditor"
DB_PASS = "GA_riscos_2025"
DB_NAME = "dicoi"

NameError: name 'project_id' is not defined

In [19]:
import pandas as pd
query = '''
SELECT cd_ficha, cd_cicl, cd_prf_gestor, tx_prf_gestor, pacr_conclusivo, analise, resultado
FROM avl_vw_ficha
JOIN avl_ficha_teste USING(cd_ficha)
LIMIT 400;
'''
df = pd.read_sql_query(query, conn_dicoi)
df.head()

/tmp/ipython-input-2229547024.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn_dicoi)


DatabaseError: Execution failed on sql: 
SELECT cd_ficha, cd_cicl, cd_prf_gestor, tx_prf_gestor, pacr_conclusivo, analise, resultado
FROM avl_vw_ficha
JOIN avl_ficha_teste USING(cd_ficha)
LIMIT 400;

server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

unable to rollback

## Divisão em lotes de 10 fichas
Evita sobrecarga na API e permite controle de processamento.

In [ ]:
def dividir_em_lotes(df, tamanho_lote=10):
    return [df[i:i + tamanho_lote] for i in range(0, len(df), tamanho_lote)]

lotes = dividir_em_lotes(df)

\## Função para formatar fichas em texto plano
Prepara os dados para envio à API.

In [ ]:
def format_fichas_avl(df_lote):
    fichas_formatadas = []
    for _, row in df_lote.iterrows():
        ficha_texto = (
            f"Ficha AVL #{row['cd_ficha']}\n"
            f"cd_ficha: {row['cd_ficha']}\n"
            f"cd_cicl: {row['cd_cicl']}\n"
            f"cd_prf_gestor: {row['cd_prf_gestor']}\n"
            f"tx_prf_gestor: {row['tx_prf_gestor']}\n"
            f"analise: {row['analise']}\n"
            f"resultado: {row['resultado']}\n"
            f"pacr_conclusivo: {row['pacr_conclusivo']}"
        )
        fichas_formatadas.append(ficha_texto)
    return fichas_formatadas

# Task
Adapt the code to connect to a PostgreSQL database via Google Cloud and replace the OpenAI API with a Google API. Provide instructions on how to configure these services to make the notebook functional.

## Configurar a conexão com o postgresql no google cloud

### Subtask:
Explicar como configurar e obter as credenciais de conexão para o banco de dados PostgreSQL hospedado no Google Cloud.


**Reasoning**:
The subtask is to explain how to configure and obtain credentials for a PostgreSQL database on Google Cloud. This requires providing instructions in a markdown cell.



**Reasoning**:
The previous command failed because a markdown block was attempted within a `code_block`. The instructions require providing the explanation in a markdown cell. I will create a markdown cell with the instructions.



## Atualizar a conexão do banco de dados no código

### Subtask:
Modificar a célula que estabelece a conexão com o banco de dados para usar as credenciais do Google Cloud e a biblioteca ou método apropriado.


**Reasoning**:
Modify the existing code cell to use the provided Google Cloud PostgreSQL credentials for the `conn_dicoi` connection, keeping the `conn_audit` connection as is.



In [24]:
# Carregamento de variáveis de ambiente
from google.colab import userdata
import os
import psycopg2

# Conexões com Bancos de Dados
# Assuming you have stored your Google Cloud PostgreSQL connection string in userdata
# Example format for public IP connection:
# "dbname=<your_database_name> user=<your_username> password=<your_password> host=<your_public_ip_address> port=5432"
# Example format for Cloud SQL Proxy connection:
# "dbname=<your_database_name> user=<your_username> password=<your_password> host=127.0.0.1 port=5432" # assuming proxy is running on localhost
conn_dicoi = psycopg2.connect(userdata.get("GOOGLE_CLOUD_POSTGRES_DICOI"))
conn_audit = psycopg2.connect(userdata.get("BASE_AUDIT"))

# Chave da API do Agente de Auditoria
#AGENTE_API_KEY = userdata.get("OPENAI_API_KEY")

SecretNotFoundError: Secret GOOGLE_CLOUD_POSTGRES_DICOI does not exist.

## Atualizar a conexão do banco de dados no código

### Subtask:
Modificar a célula que estabelece a conexão com o banco de dados para usar as credenciais do Google Cloud e a biblioteca ou método apropriado.


**Reasoning**:
Modify the code cell to use the Google Cloud PostgreSQL connection string from `userdata` and keep the `conn_audit` connection as is. Comment out the `AGENTE_API_KEY` line.

